In [1]:
import polars as pl
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import sqlite3

timeout_time = 3600

## Conncect to database
con = sqlite3.connect(os.environ["database_name"])
cur = con.cursor()

In [2]:
df = pl.read_database_uri("SELECT * FROM CaseBasicInfo WHERE Type = \'JUV\' AND ExactDOB IS NULL", os.environ["database"])

In [3]:
df.describe

<bound method DataFrame.describe of shape: (12_696, 5)
┌─────────────────────┬──────┬─────────────┬────────────────────┬──────────┐
│ Person              ┆ Type ┆ YearOfBirth ┆ CaseID             ┆ ExactDOB │
│ ---                 ┆ ---  ┆ ---         ┆ ---                ┆ ---      │
│ str                 ┆ str  ┆ i64         ┆ str                ┆ str      │
╞═════════════════════╪══════╪═════════════╪════════════════════╪══════════╡
│ HERNANDEZ,DANIEL,JR ┆ JUV  ┆ 2014        ┆ D 59 JV 14 0000434 ┆ null     │
│ ROBINSON,SHIESHA    ┆ JUV  ┆ 1998        ┆ D 01 JV 14 0001708 ┆ null     │
│ HALL,SHELIA         ┆ JUV  ┆ 1999        ┆ D 01 JV 14 0001708 ┆ null     │
│ BROWN,SHELIA        ┆ JUV  ┆ 1999        ┆ D 01 JV 14 0001708 ┆ null     │
│ …                   ┆ …    ┆ …           ┆ …                  ┆ …        │
│ ANDREWS,JOY         ┆ JUV  ┆ 1992        ┆ D 01 JV 10 9048750 ┆ null     │
│ BRITTENHAM,CHELSEA  ┆ JUV  ┆ 1991        ┆ D 01 JV 10 9048748 ┆ null     │
│ HILL,JOHN,K        

In [4]:
cService = webdriver.ChromeService(executable_path=os.environ["chromedriver"])
browser = webdriver.Chrome(service = cService)
browser.command_executor._client_config._timeout = timeout_time

for name in df["Person"]:

    browser.get(os.environ["url"])

    browser.find_element(By.ID, "party_name").send_keys(name)
    browser.find_element(By.CSS_SELECTOR, "button.btn.btn-primary[name='submit_hidden']").click()

    # Parse the HTML
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    # Extract all DOB instances using a regex

    # This regex looks for "DOB:" followed by a date in MM/DD/YYYY format
    dob_pattern = r'DOB:\s*\d{2}/\d{2}/\d{4}'
    dob_matches = list(set(re.findall(dob_pattern, soup.get_text())))

    dob_matches = '; '.join(dob_matches)

    cur.execute(f"""
            UPDATE CaseBasicInfo
            SET ExactDOB = \"{dob_matches}\"
            WHERE Person = \"{name}\" AND Type = \'JUV\'
        """)
    
    con.commit()

browser.close()